 # Tutorial 06 - Thresholding
 
 ## Dr. David C. Schedl

 Note: this tutorial is geared towards students **experienced in programming** and aims to introduce you to **Digital Imaging / Computer Vision** techniques.


 # Table of Contents

 - Thresholding
 - Binary Image Regions and their Properties


 # Initilization

 As always let's import useful libraries, first.

In [ ]:
import os
import cv2  # openCV
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, fixed, IntSlider, FloatSlider
from skimage import data
import pandas as pd

 We will work with images today. So let's download some with `curl` (the same sources as in `02_OpenCV.ipynb`).

In [ ]:
!curl -o "cat.jpg" "https://placekitten.com/320/320" --silent
!curl -o "gogh.jpg" "https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Vincent_van_Gogh_-_National_Gallery_of_Art.JPG/367px-Vincent_van_Gogh_-_National_Gallery_of_Art.JPG" --silent
!curl -o "sudoku.png" "https://raw.githubusercontent.com/opencv/opencv/4.x/samples/data/sudoku.png" --silent
!curl -o "shapes.png" "https://raw.githubusercontent.com/Digital-Media/di_cv/main/data/shapes.png" --silent

 # Thresholding

 Let's first look at our sample image showing multiple coins on a fairly uniform background.

In [ ]:
coins = data.coins()

fig = plt.figure(figsize=(12, 7), facecolor="white")
plt.subplot(1, 2, 1), plt.imshow(coins, cmap="gray")
plt.title(f"Image {coins.shape[::-1]}"), plt.xticks([]), plt.yticks([])
plt.subplot(1, 2, 2), plt.hist(coins.ravel(), bins=256, range=[0, 255])
plt.title("Histogram"), plt.yticks([])
plt.show()

 ## Global Thresholding

 Let's first look at a global thresholding approach.
 All pixels with a value above the threshold are True, all pixels with a value below the threshold are False.
 Can you come up with a threshold value that separates the coins from the background? <br>
 You can also try to blur the image. Does this help?

In [ ]:
# a slider for changing the threshold

img = data.coins()
print(img.shape)


def plot_threshold(threshold, blur=0):
    _img = img.copy()

    if blur > 0:
        _img = cv2.GaussianBlur(_img, None, blur)

    T = _img > threshold

    # plot the thresholded image and the histogram
    fig = plt.figure(figsize=(15, 7))
    plt.subplot(1, 2, 1), plt.imshow(T, cmap="gray", vmin=0, vmax=1)
    plt.title(f"Threshold ({T.sum()/T.size*100:3.2f}% pixels selected)"), plt.xticks(
        []
    ), plt.yticks([])
    plt.subplot(1, 2, 2),
    plt.hist(_img[T].ravel(), range=[0, 255], bins=256),
    plt.hist(_img[np.invert(T)].ravel(), range=[0, 255], bins=256)
    plt.show()
    # return fig


interact(
    plot_threshold,
    threshold=IntSlider(min=0, max=255, step=1, value=128),
    blur=FloatSlider(min=0, max=5, step=0.1, value=0),
)

## Automatic Thresholding (Otsu)

Let's now look at an automatic thresholding approach, where $q$ is computed based on some heuristics.


In [ ]:
img = data.coins()

# Otsu's thresholding
ret, th = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
th = th > 0 

# plot the thresholded image and the histogram
fig = plt.figure(figsize=(15, 7), facecolor="white")
plt.subplot(1, 2, 1), plt.imshow(th, cmap="gray", vmin=0, vmax=1)
plt.title(f"Automatic Threshold at {ret} ({th.sum()/th.size*100:3.2f}% pixels selected)"), plt.xticks(
    []
), plt.yticks([])
plt.subplot(1, 2, 2),
plt.hist(img[th].ravel(), range=[0, 255], bins=256),
plt.hist(img[np.invert(th)].ravel(), range=[0, 255], bins=256)
plt.show()
# return fig


### Exercise 1 📝: <a name="Exercise_1" id="Exercise_1">  </a> Otsu under the hood 

The Otsu algorithm is based on the following idea: <br>
Given a grayscale image $I$ and a threshold $q$, we can compute the **weighted between-class variance** $B(q)$ as follows:

$$
B(q) = c_{w} \cdot \sigma^2_{w}(q) + c_{b} \cdot \sigma^2_{b}(q)
$$

where $\sigma^2_{w}(q)$ is the variance of the **foreground** and $\sigma^2_{b}(q)$ is the variance of the **background** and $c_{w}$ and $c_{b}$ are the number of pixels in the **foreground** and **background**, respectively.

The **foreground** is defined as all pixels with a value greater than $q$ and the **background** as all pixels with a value less than and equal $q$. <br>

The Otsu algorithm seeks to find the threshold $q$ that minimizes the **weighted between-class variance** $B(q)$.



In [ ]:
img = data.coins()
#img = cv2.imread("cat.jpg", 0)
#img = cv2.imread("gogh.jpg", 0)

# OpenCV's adaptive thresholding for reference
ref, _ = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# calculate the global otsu threshold
def my_otsu(img):
    fn_min = np.inf
    thresh = -1

    for q in range (256):

        # apply threshold
        th = img > q

        # Todo: find the threshold that minimizes B(q)

    return thresh

print( f'Our Otsu threshold {my_otsu(img)} vs OpenCV\'s {ref}')

 ## Local Thresholding

 Let's now look at a local thresholding approach.
 In comparison to the global thresholding, the local thresholding is applied to small regions of the image.
 The region size is defined by the $w$ parameter.

### Exercise 2 📝: <a name="Exercise_2" id="Exercise_2">  </a> Local Thresholding

Try to come up with a threshold, window and blur that separates the coins from the background. <br>

In [ ]:
def adaptive_threshold(w=11, c=2, blur=0):
    img = data.coins()

    if blur > 0:
        img = cv2.GaussianBlur(img, None, blur)

    th = cv2.adaptiveThreshold(
        img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, w, c
    )

    fig = plt.figure(figsize=(15, 7), facecolor="white")
    titles = [
        "Original Image" if blur == 0 else f"Blurred Image",
        "Adaptive Mean Thresholding",
    ]
    images = [img, th]
    for i in range(len(images)):
        plt.subplot(1, 2, i + 1), plt.imshow(images[i], "gray")
        plt.title(titles[i])
        plt.xticks([]), plt.yticks([])
    plt.show()


interact(
    adaptive_threshold,
    w=IntSlider(min=3, max=255, step=2, value=51),
    c=IntSlider(min=-50, max=50, step=1, value=2),
    blur=FloatSlider(min=0, max=5, step=0.1, value=0),
)

### Another example: The Sudoko image

Let's now look at another hard example: the Sudoko image (from the slides).

In [ ]:

_img = cv2.imread('sudoku.png',0)
img = cv2.medianBlur(_img.copy(),7).copy()
ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
titles = ['Original Image', 'Global Thresholding ($q$ = 127)',
            'Adaptive Mean Thresholding']
images = [_img, th1, th2]
plt.figure(figsize=(15, 15), facecolor="white")
for i in range(len(images)):
    plt.subplot(1,len(images),i+1),
    plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

# Image Regions

Let's first reuse the coins image from above. 
And let's look at the image regions of it.

In [ ]:
# good settings for 'coins'
w,c,blur = 115, -22, 3

img = data.coins()
img = cv2.GaussianBlur(img, None, blur)

th = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, w, c
)
binary = th > 0 
plt.imshow(binary, cmap="gray", vmin=0, vmax=1)

## Labels and Simple Region Properties

Let's now look at the labels and simple region properties of the image regions, such as the area, the centroid, the bounding box, etc.

In [ ]:
# image regions

import cv2

# get binary regions of binary image and compute their properties (area, BBs, centroid)
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(binary.astype(np.uint8))
print( f"Found {len(np.unique(labels))-1} (connected) regions")

# compute the contour perimeter of each region
perimeters = np.zeros_like(stats[:,4])
for i in range(1, len(np.unique(labels))):
    perimeters[i] = cv2.arcLength(cv2.findContours((labels==i).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0][0], True)

plt.figure(figsize=(15, 7))
plt.imshow(labels, cmap="jet")
plt.show()


In [ ]:
# for each label (except the background) plot the bounding box and centroid

img = data.coins()
plt.figure(figsize=(15, 7))
plt.imshow(img, cmap="gray", vmin=0, vmax=255)

for i in range(1, len(stats)):
    x, y, w, h, area = stats[i]
    # plot the BB as rectangle
    plt.plot([x, x + w, x + w, x, x], [y, y, y + h, y + h, y], "go-")
    plt.plot(centroids[i, 0], centroids[i, 1], "b+")
    plt.text( x, y, f"#: {i} \nP: {perimeters[i]} \nA: {area}", color="black", fontsize=10, verticalalignment="top", horizontalalignment="left")

plt.show()


## Moments of Binary Image Regions

Let's now look at the moments of one image region. 
We will also use OpenCV to compute Hu's moments.

In [ ]:
region_id = 3

# First compute the moments
# regular moments (m_ij)
# central moments (mu_ij)
# normalized moments (nu_ij)
moments = cv2.moments((labels==region_id).astype(np.uint8), True)
print(moments)


# compute the Hu moments
hu_moments = cv2.HuMoments(moments)
print(hu_moments.T)

# Binary Image Regions: 5 standard shapes

In [ ]:
# a helper function to compute multiple region properties
def compute_properties(img, hu_log=True):
    """Compute properties of a binary image.
    Args:
        img (np.array): binary image
        hu_log (bool): if True, compute the log of the Hu moments
    Returns:
        dict: dictionary with properties area, perimeter, circularity, and hu moments (hu_0, hu_1, ... hu_6)
    """
    # get binary regions of binary image and compute their properties (area, BBs, centroid)
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    assert( len(stats) == 2 ) # foreground (1) and background (0)
    # compute the contour perimeter 
    perimeter = cv2.arcLength(cv2.findContours((labels==1).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0][0], True)
    circularity = 4.0*np.pi*stats[1,4]/ (.95*perimeter)**2

    simple_props = {'area': stats[1,4], 'perimeter': perimeter, 'circularity': circularity}

    # compute Hu moments 
    hu_moments = cv2.HuMoments(cv2.moments((labels==1).astype(np.uint8))).flatten()
    if hu_log:
        hu_moments = np.sign(hu_moments) * np.log(np.abs(hu_moments)) # log is only defined for positive values, thus use abs
    hu_props = {'hu_'+str(i): hu_moments[i] for i in range(len(hu_moments))}

    return dict( **simple_props, **hu_props )

In [ ]:
# load the shapes image and binarize it
img = cv2.imread("shapes.png", cv2.IMREAD_GRAYSCALE)
#img = cv2.flip(cv2.flip(img, 1), 0) # flip image
#img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST) # resize
img = (img>0).astype(np.uint8)

# get binary regions of binary image and compute their properties (area, BBs, centroid)
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img)

plt.figure(figsize=(15, 7), facecolor="white")
plt.imshow(labels, cmap="tab10", vmin=0, vmax=len(stats))
# turn off the axis
plt.axis("off")

for i in range(1, len(stats)):
    x, y, w, h, area = stats[i]
    # plot the BB as rectangle
    plt.plot([x, x + w, x + w, x, x], [y, y, y + h, y + h, y], "wo-")
    plt.plot(centroids[i, 0], centroids[i, 1], "k+")
    plt.text( centroids[i, 0], y+h+10, f"#: {i}", color="black", fontsize=12,
        verticalalignment="top", horizontalalignment="center", # turn on latex rendering 
    )


plt.show()



In [ ]:
props = []
for i in range(1, len(stats)):
    props.append( compute_properties((labels==i).astype(np.uint8)) )

# show as pandas table
df_shapes = pd.DataFrame(props, index=range(1, len(stats)))
df_shapes

# Compare/Match Shapes

Let's now look at the shape descriptions of our 5 standard shapes and how we can match them with our coins image.
First lets look at the shape descriptions of our coins.

In [ ]:
# good settings for 'coins'
w,c,blur = 115, -22, 3

img = data.coins()
img = cv2.GaussianBlur(img, None, blur)

th = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, w, c
)
binary = th > 0 

# get binary regions of binary image and compute their properties (area, BBs, centroid)
retval, labels, stats, centroids = cv2.connectedComponentsWithStats(binary.astype(np.uint8))

coin_props = []
for i in range(1, len(stats)):
    coin_props.append( compute_properties((labels==i).astype(np.uint8)) )

# show as pandas table
df_coins = pd.DataFrame(coin_props)   
df_coins

## Let's visualize the shape descriptions (in 2D)

In [ ]:
# plot the coins and the shapes on a 2D plot

x_axis = "hu_0"
y_axis = "hu_1"


plt.figure(figsize=(15, 7))
# set the axis labels
plt.xlabel(x_axis)
plt.ylabel(y_axis)

# plot the coins in blue
plt.scatter(df_coins[x_axis], df_coins[y_axis], c="b", label="coins")

# plot the shapes in red
plt.scatter(df_shapes[x_axis], df_shapes[y_axis], c="r", label="shapes")
# put the region number as label
for i in df_shapes.index:
    plt.text(df_shapes[x_axis][i], df_shapes[y_axis][i], f"#{i}", color="black", fontsize=12, verticalalignment="bottom", horizontalalignment="center")

plt.legend()
plt.show()